In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector

In [2]:
from scipy import stats

%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from dmba import classificationSummary, gainsChart

In [3]:
import os

# print the current working directory
print("Current working directory:", os.getcwd())

# change the working directory
os.chdir('C://Users//nim3091//Desktop//Ismail')

# print the new working directory
print("New working directory:", os.getcwd())

Current working directory: C:\Users\nim3091
New working directory: C:\Users\nim3091\Desktop\Ismail


In [4]:
accidents_df = pd.read_csv("project_data.csv")

In [5]:
# To visualize dataset
accidents_df

,S/N,CASENUM,REGION,URBANICITY,VE_TOTAL,PVH_INVL,PERMVIT,PERNOTMVIT,MONTH,YEAR,...,HOUR_DAYTIME,DISTRACTED,REST_USED,REST_TYP,WEATHER_IM,WEATHER_G,WEEKEND,SPEEDREL_UPD,WEEKDAY_UPD,DISTRACTED_DRIVING
0,0,2.020000e+11,West,Urban,4orMore,NoVehicle,FourOcc,NoPerson,1,2018,...,7-8AM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,Yes,Wednesday,NoDistraction
1,1,2.020000e+11,West,Urban,4orMore,NoVehicle,FourOcc,NoPerson,1,2018,...,7-8AM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,No,Wednesday,NoDistraction
2,2,2.020000e+11,West,Urban,4orMore,NoVehicle,FourOcc,NoPerson,1,2018,...,7-8AM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,No,Wednesday,NoDistraction
3,3,2.020000e+11,West,Urban,4orMore,NoVehicle,FourOcc,NoPerson,1,2018,...,7-8AM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,No,Wednesday,NoDistraction
4,4,2.020000e+11,West,Urban,Three,NoVehicle,ThreeOcc,NoPerson,1,2018,...,2-4PM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,Yes,Tuesday,NoDistraction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358957,372370,2.020000e+11,South,Urban,Two,NoVehicle,5to7Occ,NoPerson,11,2020,...,5-7PM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,No,Monday,NoDistraction
358958,372371,2.020000e+11,South,Urban,Two,NoVehicle,FourOcc,NoPerson,12,2020,...,11-12PM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,No,Monday,NoDistraction
358959,372372,2.020000e+11,South,Urban,Two,NoVehicle,FourOcc,NoPerson,12,2020,...,11-12PM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,No,Monday,NoDistraction
358960,372373,2.020000e+11,South,Urban,Two,NoVehicle,FourOcc,NoPerson,12,2020,...,11-12PM,No,Restraint,ShoulderLap,Clear,ClearCloudy,Weekday,No,Monday,NoDistraction


In [6]:
# selecting the important variables for our model manually
filterdata = accidents_df[['ALCHL_IM','DAYNIGHT','DISTRACTED','DRUGS','IMPACT1_IM','INJ_SEV_CAT','LGTCON_IM',
'MONTH','NUMOCCS','REST_MIS','REST_TYP','REST_USED','ROLLOVER','SEAT_IM','SEX_IM','SPEEDREL_UPD','TRAV_SP',
'VE_TOTAL','VSPD_LIM','VSURCOND','WEATHER_IM','WEEKDAY_UPD','YEAR']]

In [7]:
# To visualize dataset with the important variables of our model
filterdata

,ALCHL_IM,DAYNIGHT,DISTRACTED,DRUGS,IMPACT1_IM,INJ_SEV_CAT,LGTCON_IM,MONTH,NUMOCCS,REST_MIS,...,SEAT_IM,SEX_IM,SPEEDREL_UPD,TRAV_SP,VE_TOTAL,VSPD_LIM,VSURCOND,WEATHER_IM,WEEKDAY_UPD,YEAR
0,No,Daytime,No,No,Front,No,Daylight,1,One,No,...,FrontL,Male,Yes,55orLess,4orMore,56-70,Dry,Clear,Wednesday,2018
1,No,Daytime,No,No,Rear,No,Daylight,1,One,No,...,FrontL,Female,No,55orLess,4orMore,56-70,Dry,Clear,Wednesday,2018
2,No,Daytime,No,No,Rear,No,Daylight,1,One,No,...,FrontL,Male,No,StoppedVehicle,4orMore,56-70,Dry,Clear,Wednesday,2018
3,No,Daytime,No,No,Rear,No,Daylight,1,One,No,...,FrontL,Male,No,StoppedVehicle,4orMore,56-70,Dry,Clear,Wednesday,2018
4,No,Daytime,No,No,Front,Minor,Daylight,1,One,No,...,FrontL,Female,Yes,70orLess,Three,56-70,Dry,Clear,Tuesday,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358957,No,Daytime,No,No,Front,No,Dark,11,One,No,...,FrontL,Male,No,55orLess,Two,26-55,Dry,Clear,Monday,2020
358958,No,Nighttime,No,No,Front,No,Dark,12,Three,No,...,FrontL,Female,No,25orLess,Two,26-55,Dry,Clear,Monday,2020
358959,No,Nighttime,No,No,Front,No,Dark,12,Three,No,...,FrontR,Female,No,25orLess,Two,26-55,Dry,Clear,Monday,2020
358960,No,Nighttime,No,No,Front,No,Dark,12,Three,No,...,SecondR,Male,No,55orLess,Two,26-55,Dry,Clear,Monday,2020


In [8]:
# separating the year and month variables prior getting dummies
dates = filterdata[['YEAR','MONTH']]
categorical = filterdata.drop(columns=(['YEAR','MONTH']))

In [9]:
# to visualize the year and month data
dates

,YEAR,MONTH
0,2018,1
1,2018,1
2,2018,1
3,2018,1
4,2018,1
...,...,...
358957,2020,11
358958,2020,12
358959,2020,12
358960,2020,12


In [10]:
# to convert the important variables as categorical to create dummies
categorical = categorical.astype('category')
categorical

,ALCHL_IM,DAYNIGHT,DISTRACTED,DRUGS,IMPACT1_IM,INJ_SEV_CAT,LGTCON_IM,NUMOCCS,REST_MIS,REST_TYP,...,ROLLOVER,SEAT_IM,SEX_IM,SPEEDREL_UPD,TRAV_SP,VE_TOTAL,VSPD_LIM,VSURCOND,WEATHER_IM,WEEKDAY_UPD
0,No,Daytime,No,No,Front,No,Daylight,One,No,ShoulderLap,...,No,FrontL,Male,Yes,55orLess,4orMore,56-70,Dry,Clear,Wednesday
1,No,Daytime,No,No,Rear,No,Daylight,One,No,ShoulderLap,...,No,FrontL,Female,No,55orLess,4orMore,56-70,Dry,Clear,Wednesday
2,No,Daytime,No,No,Rear,No,Daylight,One,No,ShoulderLap,...,No,FrontL,Male,No,StoppedVehicle,4orMore,56-70,Dry,Clear,Wednesday
3,No,Daytime,No,No,Rear,No,Daylight,One,No,ShoulderLap,...,No,FrontL,Male,No,StoppedVehicle,4orMore,56-70,Dry,Clear,Wednesday
4,No,Daytime,No,No,Front,Minor,Daylight,One,No,ShoulderLap,...,No,FrontL,Female,Yes,70orLess,Three,56-70,Dry,Clear,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358957,No,Daytime,No,No,Front,No,Dark,One,No,ShoulderLap,...,No,FrontL,Male,No,55orLess,Two,26-55,Dry,Clear,Monday
358958,No,Nighttime,No,No,Front,No,Dark,Three,No,ShoulderLap,...,No,FrontL,Female,No,25orLess,Two,26-55,Dry,Clear,Monday
358959,No,Nighttime,No,No,Front,No,Dark,Three,No,ShoulderLap,...,No,FrontR,Female,No,25orLess,Two,26-55,Dry,Clear,Monday
358960,No,Nighttime,No,No,Front,No,Dark,Three,No,ShoulderLap,...,No,SecondR,Male,No,55orLess,Two,26-55,Dry,Clear,Monday


In [11]:
# to separate the outcome variable into the Year-Month DataFrame
dates_outcome = pd.concat([dates, categorical['INJ_SEV_CAT']],axis=1)
predictors = categorical.drop(columns=('INJ_SEV_CAT'))

In [12]:
# create a dictionary to map categorical values to numerical values for our outcome
outcome_map = {'No': 0, 'Minor': 1, 'Severe': 2}
dates_outcome['INJ_SEV_CAT'] = dates_outcome['INJ_SEV_CAT'].map(outcome_map)
dates_outcome

,YEAR,MONTH,INJ_SEV_CAT
0,2018,1,0
1,2018,1,0
2,2018,1,0
3,2018,1,0
4,2018,1,1
...,...,...,...
358957,2020,11,0
358958,2020,12,0
358959,2020,12,0
358960,2020,12,0


In [13]:
# to verify the names of our predictors
predictors.columns

Index(['ALCHL_IM', 'DAYNIGHT', 'DISTRACTED', 'DRUGS', 'IMPACT1_IM',
       'LGTCON_IM', 'NUMOCCS', 'REST_MIS', 'REST_TYP', 'REST_USED', 'ROLLOVER',
       'SEAT_IM', 'SEX_IM', 'SPEEDREL_UPD', 'TRAV_SP', 'VE_TOTAL', 'VSPD_LIM',
       'VSURCOND', 'WEATHER_IM', 'WEEKDAY_UPD'],
      dtype='object')

In [14]:
# to get dummies for our predictors and droping one (n-1 dummies rule)
filterdata_dummies = pd.get_dummies(predictors, drop_first = True)

In [15]:
# to visualize the creation of dummies
filterdata_dummies

,ALCHL_IM_Yes,DAYNIGHT_Nighttime,DISTRACTED_Yes,DRUGS_Yes,IMPACT1_IM_Left,IMPACT1_IM_LeftBack,IMPACT1_IM_LeftFront,IMPACT1_IM_NonCollision,IMPACT1_IM_PartsinMotion,IMPACT1_IM_Rear,...,WEATHER_IM_Other,WEATHER_IM_Rain,WEATHER_IM_SevereCrosswinds,WEATHER_IM_Snow,WEEKDAY_UPD_Monday,WEEKDAY_UPD_Saturday,WEEKDAY_UPD_Sunday,WEEKDAY_UPD_Thursday,WEEKDAY_UPD_Tuesday,WEEKDAY_UPD_Wednesday
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
358958,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
358959,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
358960,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [16]:
# to merge both dataframes (outcome and year-month dataframe with predictors dataframe)
complete_data = pd.concat([dates_outcome, filterdata_dummies],axis=1)
complete_data

,YEAR,MONTH,INJ_SEV_CAT,ALCHL_IM_Yes,DAYNIGHT_Nighttime,DISTRACTED_Yes,DRUGS_Yes,IMPACT1_IM_Left,IMPACT1_IM_LeftBack,IMPACT1_IM_LeftFront,...,WEATHER_IM_Other,WEATHER_IM_Rain,WEATHER_IM_SevereCrosswinds,WEATHER_IM_Snow,WEEKDAY_UPD_Monday,WEEKDAY_UPD_Saturday,WEEKDAY_UPD_Sunday,WEEKDAY_UPD_Thursday,WEEKDAY_UPD_Tuesday,WEEKDAY_UPD_Wednesday
0,2018,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2018,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2018,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2018,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2018,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358957,2020,11,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
358958,2020,12,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
358959,2020,12,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
358960,2020,12,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
# to determine the types of our predictors
complete_data.dtypes

YEAR                        int64
MONTH                       int64
INJ_SEV_CAT              category
ALCHL_IM_Yes                uint8
DAYNIGHT_Nighttime          uint8
                           ...   
WEEKDAY_UPD_Saturday        uint8
WEEKDAY_UPD_Sunday          uint8
WEEKDAY_UPD_Thursday        uint8
WEEKDAY_UPD_Tuesday         uint8
WEEKDAY_UPD_Wednesday       uint8
Length: 86, dtype: object

In [18]:
# to verify if the dataset has all the variables
print(complete_data.head())

   YEAR  MONTH INJ_SEV_CAT  ALCHL_IM_Yes  DAYNIGHT_Nighttime  DISTRACTED_Yes  \
0  2018      1           0             0                   0               0   
1  2018      1           0             0                   0               0   
2  2018      1           0             0                   0               0   
3  2018      1           0             0                   0               0   
4  2018      1           1             0                   0               0   

   DRUGS_Yes  IMPACT1_IM_Left  IMPACT1_IM_LeftBack  IMPACT1_IM_LeftFront  ...  \
0          0                0                    0                     0  ...   
1          0                0                    0                     0  ...   
2          0                0                    0                     0  ...   
3          0                0                    0                     0  ...   
4          0                0                    0                     0  ...   

   WEATHER_IM_Other  WEATHER_IM_

In [19]:
# To eliminate the 2018 and 2019 records
reduce_df = complete_data[complete_data['YEAR'] == 2020]
print(reduce_df.head())

        YEAR  MONTH INJ_SEV_CAT  ALCHL_IM_Yes  DAYNIGHT_Nighttime  \
237439  2020      1           0             0                   0   
237440  2020      1           0             0                   0   
237441  2020      1           1             0                   1   
237442  2020      1           0             0                   0   
237443  2020      1           0             0                   0   

        DISTRACTED_Yes  DRUGS_Yes  IMPACT1_IM_Left  IMPACT1_IM_LeftBack  \
237439               0          0                1                    0   
237440               0          0                0                    0   
237441               1          0                0                    0   
237442               0          0                0                    0   
237443               0          0                0                    0   

        IMPACT1_IM_LeftFront  ...  WEATHER_IM_Other  WEATHER_IM_Rain  \
237439                     0  ...                 0           

In [20]:
### to define 3 dataframes according to the different stages of the pandemic
before_lockdown = (reduce_df['MONTH'] < 3)
lockdown = ((reduce_df['MONTH'] >= 3) & (reduce_df['MONTH'] <= 5))
post_lockdown = reduce_df['MONTH'] > 5

# create three separate DataFrames based on the conditions
df_before_lockdown = reduce_df[before_lockdown]
df_lockdown = reduce_df[lockdown]
df_post_lockdown = reduce_df[post_lockdown]

# display the resulting DataFrames
print('Before lockdown:')
print(df_before_lockdown.head())
print('\nLockdown:')
print(df_lockdown.head())
print('\nPost lockdown:')
print(df_post_lockdown.head())

Before lockdown:
        YEAR  MONTH INJ_SEV_CAT  ALCHL_IM_Yes  DAYNIGHT_Nighttime  \
237439  2020      1           0             0                   0   
237440  2020      1           0             0                   0   
237441  2020      1           1             0                   1   
237442  2020      1           0             0                   0   
237443  2020      1           0             0                   0   

        DISTRACTED_Yes  DRUGS_Yes  IMPACT1_IM_Left  IMPACT1_IM_LeftBack  \
237439               0          0                1                    0   
237440               0          0                0                    0   
237441               1          0                0                    0   
237442               0          0                0                    0   
237443               0          0                0                    0   

        IMPACT1_IM_LeftFront  ...  WEATHER_IM_Other  WEATHER_IM_Rain  \
237439                     0  ...            

In [21]:
df_before_lockdown.columns


Index(['YEAR', 'MONTH', 'INJ_SEV_CAT', 'ALCHL_IM_Yes', 'DAYNIGHT_Nighttime',
       'DISTRACTED_Yes', 'DRUGS_Yes', 'IMPACT1_IM_Left', 'IMPACT1_IM_LeftBack',
       'IMPACT1_IM_LeftFront', 'IMPACT1_IM_NonCollision',
       'IMPACT1_IM_PartsinMotion', 'IMPACT1_IM_Rear', 'IMPACT1_IM_Right',
       'IMPACT1_IM_RightBack', 'IMPACT1_IM_RightFront', 'IMPACT1_IM_Top',
       'IMPACT1_IM_Undercarriage', 'LGTCON_IM_Dawn', 'LGTCON_IM_Daylight',
       'LGTCON_IM_Dusk', 'NUMOCCS_9-May', 'NUMOCCS_FourorFive', 'NUMOCCS_One',
       'NUMOCCS_Three', 'NUMOCCS_Two', 'REST_MIS_NotVehicle', 'REST_MIS_Yes',
       'REST_TYP_Helmet', 'REST_TYP_NoRest', 'REST_TYP_NotApplicable',
       'REST_TYP_OtherRest', 'REST_TYP_ShoulderLap', 'REST_USED_Restraint',
       'ROLLOVER_Yes', 'SEAT_IM_FrontL', 'SEAT_IM_FrontM', 'SEAT_IM_FrontOth',
       'SEAT_IM_FrontR', 'SEAT_IM_OtherPassenger', 'SEAT_IM_SecondL',
       'SEAT_IM_SecondM', 'SEAT_IM_SecondOth', 'SEAT_IM_SecondR',
       'SEAT_IM_SleeperSection', 'SEAT_IM_T

## Best predictors - before lockdown period

In [22]:
predictors1 = ['YEAR', 'MONTH', 'INJ_SEV_CAT', 'ALCHL_IM_Yes', 'DAYNIGHT_Nighttime',
       'DISTRACTED_Yes', 'DRUGS_Yes', 'IMPACT1_IM_Left', 'IMPACT1_IM_LeftBack',
       'IMPACT1_IM_LeftFront', 'IMPACT1_IM_NonCollision',
       'IMPACT1_IM_PartsinMotion', 'IMPACT1_IM_Rear', 'IMPACT1_IM_Right',
       'IMPACT1_IM_RightBack', 'IMPACT1_IM_RightFront', 'IMPACT1_IM_Top',
       'IMPACT1_IM_Undercarriage', 'LGTCON_IM_Dawn', 'LGTCON_IM_Daylight',
       'LGTCON_IM_Dusk', 'NUMOCCS_9-May', 'NUMOCCS_FourorFive', 'NUMOCCS_One',
       'NUMOCCS_Three', 'NUMOCCS_Two', 'REST_MIS_NotVehicle', 'REST_MIS_Yes',
       'REST_TYP_Helmet', 'REST_TYP_NoRest', 'REST_TYP_NotApplicable',
       'REST_TYP_OtherRest', 'REST_TYP_ShoulderLap', 'REST_USED_Restraint',
       'ROLLOVER_Yes', 'SEAT_IM_FrontL', 'SEAT_IM_FrontM', 'SEAT_IM_FrontOth',
       'SEAT_IM_FrontR', 'SEAT_IM_OtherPassenger', 'SEAT_IM_SecondL',
       'SEAT_IM_SecondM', 'SEAT_IM_SecondOth', 'SEAT_IM_SecondR',
       'SEAT_IM_SleeperSection', 'SEAT_IM_ThirdRow',
       'SEAT_IM_TrailerorExterior', 'SEX_IM_Male', 'SPEEDREL_UPD_Racing',
       'SPEEDREL_UPD_Yes', 'TRAV_SP_55orLess', 'TRAV_SP_70orLess',
       'TRAV_SP_90Plus', 'TRAV_SP_90orLess', 'TRAV_SP_StoppedVehicle',
       'VE_TOTAL_One', 'VE_TOTAL_Three', 'VE_TOTAL_Two', 'VSPD_LIM_26-55',
       'VSPD_LIM_56-70', 'VSPD_LIM_71-90', 'VSPD_LIM_NonTraffic',
       'VSURCOND_Ice', 'VSURCOND_MudDirt', 'VSURCOND_NonTrafficway',
       'VSURCOND_Oil', 'VSURCOND_Other', 'VSURCOND_Sand', 'VSURCOND_Slush',
       'VSURCOND_Snow', 'VSURCOND_Water', 'VSURCOND_Wet', 'WEATHER_IM_Clear',
       'WEATHER_IM_Cloudy', 'WEATHER_IM_Fog', 'WEATHER_IM_Hail',
       'WEATHER_IM_Other', 'WEATHER_IM_Rain', 'WEATHER_IM_SevereCrosswinds',
       'WEATHER_IM_Snow', 'WEEKDAY_UPD_Monday', 'WEEKDAY_UPD_Saturday',
       'WEEKDAY_UPD_Sunday', 'WEEKDAY_UPD_Thursday', 'WEEKDAY_UPD_Tuesday',
       'WEEKDAY_UPD_Wednesday']

backward = ['ALCHL_IM_Yes', 'DISTRACTED_Yes', 'DRUGS_Yes',
       'IMPACT1_IM_NonCollision', 'IMPACT1_IM_Rear', 'IMPACT1_IM_RightFront',
       'LGTCON_IM_Daylight', 'LGTCON_IM_Dusk', 'NUMOCCS_One', 'NUMOCCS_Three',
       'NUMOCCS_Two', 'REST_MIS_NotVehicle', 'REST_TYP_NotApplicable',
       'ROLLOVER_Yes', 'SEAT_IM_FrontL', 'SEAT_IM_FrontR', 'SEAT_IM_SecondR',
       'SPEEDREL_UPD_Yes', 'TRAV_SP_55orLess', 'TRAV_SP_70orLess',
       'TRAV_SP_90orLess', 'VE_TOTAL_One', 'VE_TOTAL_Three', 'VE_TOTAL_Two',
       'VSPD_LIM_26-55', 'VSPD_LIM_56-70', 'VSURCOND_Ice',
       'VSURCOND_NonTrafficway', 'WEATHER_IM_SevereCrosswinds',
       'WEEKDAY_UPD_Monday', 'WEEKDAY_UPD_Saturday', 'WEEKDAY_UPD_Sunday',
       'WEEKDAY_UPD_Thursday']

forward = ['ALCHL_IM_Yes', 'DAYNIGHT_Nighttime', 'IMPACT1_IM_LeftFront',
       'IMPACT1_IM_PartsinMotion', 'IMPACT1_IM_RightBack',
       'IMPACT1_IM_RightFront', 'IMPACT1_IM_Undercarriage', 'LGTCON_IM_Dawn',
       'LGTCON_IM_Daylight', 'LGTCON_IM_Dusk', 'REST_MIS_NotVehicle',
       'REST_TYP_Helmet', 'REST_TYP_NoRest', 'REST_TYP_NotApplicable',
       'REST_TYP_OtherRest', 'REST_USED_Restraint', 'ROLLOVER_Yes',
       'SEAT_IM_FrontM', 'SEAT_IM_FrontOth', 'SEAT_IM_OtherPassenger',
       'SEAT_IM_SecondL', 'SEAT_IM_SecondM', 'SEAT_IM_SecondOth',
       'SEAT_IM_SecondR', 'SEAT_IM_SleeperSection', 'SEAT_IM_ThirdRow',
       'SEAT_IM_TrailerorExterior', 'SPEEDREL_UPD_Racing', 'TRAV_SP_70orLess',
       'TRAV_SP_90Plus', 'TRAV_SP_90orLess', 'VSPD_LIM_56-70',
       'VSPD_LIM_NonTraffic', 'VSURCOND_Ice', 'VSURCOND_MudDirt',
       'VSURCOND_NonTrafficway', 'VSURCOND_Oil', 'VSURCOND_Other',
       'VSURCOND_Slush', 'VSURCOND_Snow', 'VSURCOND_Wet', 'WEATHER_IM_Clear',
       'WEATHER_IM_Cloudy', 'WEATHER_IM_Fog', 'WEATHER_IM_Other',
       'WEATHER_IM_Snow', 'WEEKDAY_UPD_Monday', 'WEEKDAY_UPD_Saturday']

stepwise = ['ALCHL_IM_Yes', 'DISTRACTED_Yes', 'DRUGS_Yes', 'IMPACT1_IM_Left',
       'IMPACT1_IM_LeftBack', 'IMPACT1_IM_NonCollision', 'IMPACT1_IM_Rear',
       'IMPACT1_IM_Right', 'IMPACT1_IM_RightBack', 'IMPACT1_IM_RightFront',
       'LGTCON_IM_Daylight', 'LGTCON_IM_Dusk', 'NUMOCCS_One', 'NUMOCCS_Two',
       'REST_MIS_NotVehicle', 'REST_TYP_NoRest', 'REST_TYP_NotApplicable',
       'ROLLOVER_Yes', 'SEAT_IM_FrontL', 'SPEEDREL_UPD_Yes',
       'TRAV_SP_55orLess', 'TRAV_SP_70orLess', 'TRAV_SP_90orLess',
       'VE_TOTAL_One', 'VE_TOTAL_Three', 'VE_TOTAL_Two', 'VSPD_LIM_26-55',
       'VSPD_LIM_56-70', 'VSPD_LIM_71-90', 'VSURCOND_NonTrafficway',
       'VSURCOND_Oil', 'VSURCOND_Snow', 'WEATHER_IM_Clear', 'WEATHER_IM_Rain',
       'WEATHER_IM_Snow', 'WEEKDAY_UPD_Saturday', 'WEEKDAY_UPD_Thursday',
       'WEEKDAY_UPD_Tuesday', 'WEEKDAY_UPD_Wednesday']

# create a list comprehension for each smaller list
appearances_1 = [name for name in predictors1 if name in backward]
appearances_2 = [name for name in predictors1 if name in forward]
appearances_3 = [name for name in predictors1 if name in stepwise]

# find the names that appear in all three lists
best_predictors_bl = list(set(appearances_1) & set(appearances_2) & set(appearances_3))

# print the results
print(f"The names that appear in all three lists are: {best_predictors_bl}")

The names that appear in all three lists are: ['VSURCOND_NonTrafficway', 'TRAV_SP_70orLess', 'VSPD_LIM_56-70', 'LGTCON_IM_Daylight', 'TRAV_SP_90orLess', 'ALCHL_IM_Yes', 'ROLLOVER_Yes', 'REST_MIS_NotVehicle', 'REST_TYP_NotApplicable', 'IMPACT1_IM_RightFront', 'LGTCON_IM_Dusk', 'WEEKDAY_UPD_Saturday']


# Final Model - Before Lockdown Period

In [23]:
# creation of the train and test data for the selection process for the before lockdown period. 
X_bl = df_before_lockdown.drop(columns = (['INJ_SEV_CAT', 'MONTH', 'YEAR']))
y_bl = df_before_lockdown['INJ_SEV_CAT']

x_trainbl, x_testbl, y_trainbl, y_testbl = train_test_split(X_bl,y_bl,test_size=0.3,random_state=5000)

In [24]:
## running the model
x_trainbl = x_trainbl[best_predictors_bl]

model = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=10000)
# define the model evaluation procedure
model.fit(x_trainbl,y_trainbl)

LogisticRegression(max_iter=10000, multi_class='multinomial')

In [25]:
# getting coefficients and p-values for analysis
coef = model.coef_
pval = 2*(1 - stats.norm.cdf(abs(coef))) # assuming a two-tailed test

# print coefficients and p-values
for i in range(len(x_trainbl.columns)):
    print(x_trainbl.columns[i], "Coefficient:", coef[0][i], "P-value:", pval[0][i])

VSURCOND_NonTrafficway Coefficient: 0.5227215380491346 P-value: 0.6011680477593926
TRAV_SP_70orLess Coefficient: -0.10658854825830362 P-value: 0.9151154039911487
VSPD_LIM_56-70 Coefficient: -0.001496613873742621 P-value: 0.9988058753424347
LGTCON_IM_Daylight Coefficient: 0.11812191903508631 P-value: 0.905971056155316
TRAV_SP_90orLess Coefficient: -0.347593105157049 P-value: 0.7281457850663409
ALCHL_IM_Yes Coefficient: -0.526072642746682 P-value: 0.5988377356623267
ROLLOVER_Yes Coefficient: -1.298628037673839 P-value: 0.1940716111101881
REST_MIS_NotVehicle Coefficient: 0.005378630528407517 P-value: 0.9957084944351013
REST_TYP_NotApplicable Coefficient: -1.4697384565161986 P-value: 0.14163260302450542
IMPACT1_IM_RightFront Coefficient: 0.14986615848054477 P-value: 0.8808702117452705
LGTCON_IM_Dusk Coefficient: 0.0749640589896013 P-value: 0.9402433081699193
WEEKDAY_UPD_Saturday Coefficient: 0.036435203439559756 P-value: 0.9709353445196198


In [26]:
# create a pandas DataFrame with variable names, coefficients, and p-values
df = pd.DataFrame({'Variable': x_trainbl.columns, 'Coefficient': coef[0], 'P-value': pval[0]})

# print the DataFrame
print(df)

                  Variable  Coefficient   P-value
0   VSURCOND_NonTrafficway     0.522722  0.601168
1         TRAV_SP_70orLess    -0.106589  0.915115
2           VSPD_LIM_56-70    -0.001497  0.998806
3       LGTCON_IM_Daylight     0.118122  0.905971
4         TRAV_SP_90orLess    -0.347593  0.728146
5             ALCHL_IM_Yes    -0.526073  0.598838
6             ROLLOVER_Yes    -1.298628  0.194072
7      REST_MIS_NotVehicle     0.005379  0.995708
8   REST_TYP_NotApplicable    -1.469738  0.141633
9    IMPACT1_IM_RightFront     0.149866  0.880870
10          LGTCON_IM_Dusk     0.074964  0.940243
11    WEEKDAY_UPD_Saturday     0.036435  0.970935


In [27]:
## to test the model accuracy
x_testbl = x_testbl[best_predictors_bl]

y_predictbl = model.predict(x_testbl)
accuracy_score(y_testbl,y_predictbl)


0.7353101691831908

In [28]:
## to test the model accuracy with the training data to avoid overfitting
y_predict_trainbl = model.predict(x_trainbl)
accuracy_score(y_trainbl,y_predict_trainbl)

0.735885839051778

In [31]:
classificationSummary(y_testbl,y_predictbl, class_names=labels) 

Confusion Matrix (Accuracy 0.7353)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                     3857                109                 14
No severe injuries               1034                160                 14
Severe injuries                   209                 75                 25


In [30]:
# define class labels
labels = ['No injuries      ', 'No severe injuries', 'Severe injuries   ']

# create confusion matrix
cm = confusion_matrix(y_testbl, y_predictbl)

# print confusion matrix with labels
print('Confusion matrix:')
print(cm)

# print confusion matrix with labels and formatting
print('\nConfusion matrix (with labels):')
print('{:<19s}{:>20s}{:>15s}{:>20s}'.format('', *labels))
for i, label in enumerate(labels):
    print('{:<15s}{:>18d}{:>19d}{:>20d}'.format(label, *cm[i]))

Confusion matrix:
[[3857  109   14]
 [1034  160   14]
 [ 209   75   25]]

Confusion matrix (with labels):
                      No injuries      No severe injuries  Severe injuries   
No injuries                    3857                109                  14
No severe injuries              1034                160                  14
Severe injuries                  209                 75                  25


## Naive Bayes Model

In [32]:
# run naive Bayes
injury_nb = MultinomialNB(alpha=0.01)
injury_nb.fit(x_trainbl,y_trainbl)

# predict probabilities
predProb_train = injury_nb.predict_proba(x_trainbl)
predProb_valid = injury_nb.predict_proba(x_testbl)

# predict class membership
y_valid_pred = injury_nb.predict(x_testbl)
y_train_pred = injury_nb.predict(x_trainbl)

In [33]:
classificationSummary(y_trainbl, y_train_pred, class_names=labels) 

print()

classificationSummary(y_testbl, y_valid_pred, class_names=labels) 

Confusion Matrix (Accuracy 0.7340)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                     9024                209                 49
No severe injuries               2443                275                 98
Severe injuries                   460                152                114

Confusion Matrix (Accuracy 0.7335)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                     3856                 90                 34
No severe injuries               1036                142                 30
Severe injuries                   205                 70                 34


## Best predictors - Lockdown period

In [34]:
predictors1 = ['YEAR', 'MONTH', 'INJ_SEV_CAT', 'ALCHL_IM_Yes', 'DAYNIGHT_Nighttime',
       'DISTRACTED_Yes', 'DRUGS_Yes', 'IMPACT1_IM_Left', 'IMPACT1_IM_LeftBack',
       'IMPACT1_IM_LeftFront', 'IMPACT1_IM_NonCollision',
       'IMPACT1_IM_PartsinMotion', 'IMPACT1_IM_Rear', 'IMPACT1_IM_Right',
       'IMPACT1_IM_RightBack', 'IMPACT1_IM_RightFront', 'IMPACT1_IM_Top',
       'IMPACT1_IM_Undercarriage', 'LGTCON_IM_Dawn', 'LGTCON_IM_Daylight',
       'LGTCON_IM_Dusk', 'NUMOCCS_9-May', 'NUMOCCS_FourorFive', 'NUMOCCS_One',
       'NUMOCCS_Three', 'NUMOCCS_Two', 'REST_MIS_NotVehicle', 'REST_MIS_Yes',
       'REST_TYP_Helmet', 'REST_TYP_NoRest', 'REST_TYP_NotApplicable',
       'REST_TYP_OtherRest', 'REST_TYP_ShoulderLap', 'REST_USED_Restraint',
       'ROLLOVER_Yes', 'SEAT_IM_FrontL', 'SEAT_IM_FrontM', 'SEAT_IM_FrontOth',
       'SEAT_IM_FrontR', 'SEAT_IM_OtherPassenger', 'SEAT_IM_SecondL',
       'SEAT_IM_SecondM', 'SEAT_IM_SecondOth', 'SEAT_IM_SecondR',
       'SEAT_IM_SleeperSection', 'SEAT_IM_ThirdRow',
       'SEAT_IM_TrailerorExterior', 'SEX_IM_Male', 'SPEEDREL_UPD_Racing',
       'SPEEDREL_UPD_Yes', 'TRAV_SP_55orLess', 'TRAV_SP_70orLess',
       'TRAV_SP_90Plus', 'TRAV_SP_90orLess', 'TRAV_SP_StoppedVehicle',
       'VE_TOTAL_One', 'VE_TOTAL_Three', 'VE_TOTAL_Two', 'VSPD_LIM_26-55',
       'VSPD_LIM_56-70', 'VSPD_LIM_71-90', 'VSPD_LIM_NonTraffic',
       'VSURCOND_Ice', 'VSURCOND_MudDirt', 'VSURCOND_NonTrafficway',
       'VSURCOND_Oil', 'VSURCOND_Other', 'VSURCOND_Sand', 'VSURCOND_Slush',
       'VSURCOND_Snow', 'VSURCOND_Water', 'VSURCOND_Wet', 'WEATHER_IM_Clear',
       'WEATHER_IM_Cloudy', 'WEATHER_IM_Fog', 'WEATHER_IM_Hail',
       'WEATHER_IM_Other', 'WEATHER_IM_Rain', 'WEATHER_IM_SevereCrosswinds',
       'WEATHER_IM_Snow', 'WEEKDAY_UPD_Monday', 'WEEKDAY_UPD_Saturday',
       'WEEKDAY_UPD_Sunday', 'WEEKDAY_UPD_Thursday', 'WEEKDAY_UPD_Tuesday',
       'WEEKDAY_UPD_Wednesday']

backward = ['ALCHL_IM_Yes', 'DISTRACTED_Yes', 'DRUGS_Yes', 'IMPACT1_IM_Left',
       'IMPACT1_IM_NonCollision', 'IMPACT1_IM_Rear', 'IMPACT1_IM_Right',
       'IMPACT1_IM_RightFront', 'IMPACT1_IM_Undercarriage',
       'LGTCON_IM_Daylight', 'LGTCON_IM_Dusk', 'NUMOCCS_FourorFive',
       'NUMOCCS_One', 'REST_TYP_NotApplicable', 'REST_USED_Restraint',
       'ROLLOVER_Yes', 'SEAT_IM_FrontL', 'SEAT_IM_FrontM', 'SEAT_IM_FrontR',
       'SEAT_IM_OtherPassenger', 'SEAT_IM_SecondOth', 'SEAT_IM_SleeperSection',
       'SEAT_IM_TrailerorExterior', 'SEX_IM_Male', 'SPEEDREL_UPD_Yes',
       'TRAV_SP_55orLess', 'TRAV_SP_90orLess', 'TRAV_SP_StoppedVehicle',
       'VE_TOTAL_One', 'VE_TOTAL_Three', 'VSPD_LIM_26-55', 'VSPD_LIM_56-70',
       'VSPD_LIM_NonTraffic', 'VSURCOND_Other', 'VSURCOND_Water',
       'WEATHER_IM_Cloudy', 'WEATHER_IM_SevereCrosswinds', 'WEATHER_IM_Snow',
       'WEEKDAY_UPD_Monday', 'WEEKDAY_UPD_Sunday', 'WEEKDAY_UPD_Tuesday',
       'WEEKDAY_UPD_Wednesday']

forward = ['ALCHL_IM_Yes', 'IMPACT1_IM_Left', 'IMPACT1_IM_LeftBack',
       'IMPACT1_IM_LeftFront', 'IMPACT1_IM_PartsinMotion',
       'IMPACT1_IM_Undercarriage', 'REST_TYP_Helmet', 'REST_TYP_NoRest',
       'REST_TYP_NotApplicable', 'REST_TYP_OtherRest', 'REST_USED_Restraint',
       'ROLLOVER_Yes', 'SEAT_IM_OtherPassenger', 'SEAT_IM_SecondL',
       'SEAT_IM_SleeperSection', 'SEAT_IM_ThirdRow',
       'SEAT_IM_TrailerorExterior', 'VSPD_LIM_56-70', 'VSURCOND_NonTrafficway',
       'VSURCOND_Other', 'VSURCOND_Water', 'VSURCOND_Wet', 'WEATHER_IM_Other',
       'WEATHER_IM_SevereCrosswinds']

stepwise = ['ALCHL_IM_Yes', 'DISTRACTED_Yes', 'DRUGS_Yes',
       'IMPACT1_IM_NonCollision', 'IMPACT1_IM_Right', 'IMPACT1_IM_RightFront',
       'IMPACT1_IM_Undercarriage', 'LGTCON_IM_Daylight', 'LGTCON_IM_Dusk',
       'NUMOCCS_One', 'REST_MIS_Yes', 'REST_TYP_Helmet',
       'REST_TYP_NotApplicable', 'REST_USED_Restraint', 'ROLLOVER_Yes',
       'SEAT_IM_OtherPassenger', 'SEAT_IM_SecondR',
       'SEAT_IM_TrailerorExterior', 'SEX_IM_Male', 'SPEEDREL_UPD_Yes',
       'TRAV_SP_55orLess', 'TRAV_SP_90Plus', 'TRAV_SP_90orLess',
       'TRAV_SP_StoppedVehicle', 'VE_TOTAL_One', 'VE_TOTAL_Three',
       'VSPD_LIM_26-55', 'VSPD_LIM_56-70', 'VSURCOND_Other',
       'WEATHER_IM_Clear', 'WEATHER_IM_Cloudy', 'WEATHER_IM_Other',
       'WEEKDAY_UPD_Monday', 'WEEKDAY_UPD_Tuesday', 'WEEKDAY_UPD_Wednesday']

# create a list comprehension for each smaller list
appearances_1 = [name for name in predictors1 if name in backward]
appearances_2 = [name for name in predictors1 if name in forward]
appearances_3 = [name for name in predictors1 if name in stepwise]

# find the names that appear in all three lists
best_predictors_lo = list(set(appearances_1) & set(appearances_2) & set(appearances_3))

# print the results
print(f"The names that appear in all three lists are: {best_predictors_bl}")

The names that appear in all three lists are: ['VSURCOND_NonTrafficway', 'TRAV_SP_70orLess', 'VSPD_LIM_56-70', 'LGTCON_IM_Daylight', 'TRAV_SP_90orLess', 'ALCHL_IM_Yes', 'ROLLOVER_Yes', 'REST_MIS_NotVehicle', 'REST_TYP_NotApplicable', 'IMPACT1_IM_RightFront', 'LGTCON_IM_Dusk', 'WEEKDAY_UPD_Saturday']


# Final Model - Lockdown Period

In [35]:
# creation of the train and test data for the selection process for the before lockdown period. 
X_lo = df_lockdown.drop(columns = (['INJ_SEV_CAT', 'MONTH', 'YEAR']))
y_lo = df_lockdown['INJ_SEV_CAT']

x_trainlo, x_testlo, y_trainlo, y_testlo = train_test_split(X_lo,y_lo,test_size=0.3,random_state=5000)

In [36]:
## running the model
x_trainlo = x_trainlo[best_predictors_lo]

model = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=10000)
# define the model evaluation procedure
model.fit(x_trainlo,y_trainlo)

LogisticRegression(max_iter=10000, multi_class='multinomial')

In [37]:
# getting coefficients and p-values for analysis
coef = model.coef_
pval = 2*(1 - stats.norm.cdf(abs(coef))) # assuming a two-tailed test

# print coefficients and p-values
for i in range(len(x_trainlo.columns)):
    print(x_trainlo.columns[i], "Coefficient:", coef[0][i], "P-value:", pval[0][i])

VSPD_LIM_56-70 Coefficient: -0.10827896746771776 P-value: 0.9137744060424948
IMPACT1_IM_Undercarriage Coefficient: 0.16371400137273237 P-value: 0.8699562947678834
ALCHL_IM_Yes Coefficient: -0.5038665137159212 P-value: 0.6143551833766847
REST_USED_Restraint Coefficient: 0.4162224100297565 P-value: 0.6772472626945834
ROLLOVER_Yes Coefficient: -1.404610251699303 P-value: 0.16013720647748197
REST_TYP_NotApplicable Coefficient: -1.7064306311415478 P-value: 0.08792793156893897
VSURCOND_Other Coefficient: -0.09768362635513028 P-value: 0.9221835176443853
SEAT_IM_OtherPassenger Coefficient: 0.14553383642435067 P-value: 0.8842894025205472
SEAT_IM_TrailerorExterior Coefficient: -0.3277483728104422 P-value: 0.7431019254644118


In [38]:
# create a pandas DataFrame with variable names, coefficients, and p-values
df = pd.DataFrame({'Variable': x_trainlo.columns, 'Coefficient': coef[0], 'P-value': pval[0]})

# print the DataFrame
print(df)

                    Variable  Coefficient   P-value
0             VSPD_LIM_56-70    -0.108279  0.913774
1   IMPACT1_IM_Undercarriage     0.163714  0.869956
2               ALCHL_IM_Yes    -0.503867  0.614355
3        REST_USED_Restraint     0.416222  0.677247
4               ROLLOVER_Yes    -1.404610  0.160137
5     REST_TYP_NotApplicable    -1.706431  0.087928
6             VSURCOND_Other    -0.097684  0.922184
7     SEAT_IM_OtherPassenger     0.145534  0.884289
8  SEAT_IM_TrailerorExterior    -0.327748  0.743102


In [39]:
## to test the model accuracy
x_testlo = x_testlo[best_predictors_lo]

y_predictlo = model.predict(x_testlo)
accuracy_score(y_testlo,y_predictlo)


0.7033105433609212

In [40]:
## to test the model accuracy with the training data to avoid overfitting
y_predict_trainlo = model.predict(x_trainlo)
accuracy_score(y_trainlo,y_predict_trainlo)

0.7195187784375723

In [41]:
classificationSummary(y_testlo,y_predictlo, class_names=labels) 

Confusion Matrix (Accuracy 0.7033)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                     3571                137                 19
No severe injuries               1111                265                 28
Severe injuries                   191                163                 73


In [42]:
# define class labels
labels = ['No injuries      ', 'No severe injuries', 'Severe injuries   ']

# create confusion matrix
cm = confusion_matrix(y_testlo, y_predictlo)

# print confusion matrix with labels
print('Confusion matrix:')
print(cm)

# print confusion matrix with labels and formatting
print('\nConfusion matrix (with labels):')
print('{:<19s}{:>20s}{:>15s}{:>20s}'.format('', *labels))
for i, label in enumerate(labels):
    print('{:<15s}{:>18d}{:>19d}{:>20d}'.format(label, *cm[i]))

Confusion matrix:
[[3571  137   19]
 [1111  265   28]
 [ 191  163   73]]

Confusion matrix (with labels):
                      No injuries      No severe injuries  Severe injuries   
No injuries                    3571                137                  19
No severe injuries              1111                265                  28
Severe injuries                  191                163                  73


## Naive Bayes Model

In [43]:
# run naive Bayes
injury_nb = MultinomialNB(alpha=0.01)
injury_nb.fit(x_trainlo,y_trainlo)

# predict probabilities
predProb_train = injury_nb.predict_proba(x_trainlo)
predProb_valid = injury_nb.predict_proba(x_testlo)

# predict class membership
y_valid_pred = injury_nb.predict(x_testlo)
y_train_pred = injury_nb.predict(x_trainlo)

In [44]:
classificationSummary(y_trainlo, y_train_pred, class_names=labels) 

print()

classificationSummary(y_testlo, y_valid_pred, class_names=labels) 

Confusion Matrix (Accuracy 0.7162)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                     8536                353                  3
No severe injuries               2392                721                  7
Severe injuries                   484                441                 30

Confusion Matrix (Accuracy 0.6963)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                     3571                153                  3
No severe injuries               1111                289                  4
Severe injuries                   191                226                 10


# Best Predictors - Post Lockdown Period

In [45]:
predictors1 = ['YEAR', 'MONTH', 'INJ_SEV_CAT', 'ALCHL_IM_Yes', 'DAYNIGHT_Nighttime',
       'DISTRACTED_Yes', 'DRUGS_Yes', 'IMPACT1_IM_Left', 'IMPACT1_IM_LeftBack',
       'IMPACT1_IM_LeftFront', 'IMPACT1_IM_NonCollision',
       'IMPACT1_IM_PartsinMotion', 'IMPACT1_IM_Rear', 'IMPACT1_IM_Right',
       'IMPACT1_IM_RightBack', 'IMPACT1_IM_RightFront', 'IMPACT1_IM_Top',
       'IMPACT1_IM_Undercarriage', 'LGTCON_IM_Dawn', 'LGTCON_IM_Daylight',
       'LGTCON_IM_Dusk', 'NUMOCCS_9-May', 'NUMOCCS_FourorFive', 'NUMOCCS_One',
       'NUMOCCS_Three', 'NUMOCCS_Two', 'REST_MIS_NotVehicle', 'REST_MIS_Yes',
       'REST_TYP_Helmet', 'REST_TYP_NoRest', 'REST_TYP_NotApplicable',
       'REST_TYP_OtherRest', 'REST_TYP_ShoulderLap', 'REST_USED_Restraint',
       'ROLLOVER_Yes', 'SEAT_IM_FrontL', 'SEAT_IM_FrontM', 'SEAT_IM_FrontOth',
       'SEAT_IM_FrontR', 'SEAT_IM_OtherPassenger', 'SEAT_IM_SecondL',
       'SEAT_IM_SecondM', 'SEAT_IM_SecondOth', 'SEAT_IM_SecondR',
       'SEAT_IM_SleeperSection', 'SEAT_IM_ThirdRow',
       'SEAT_IM_TrailerorExterior', 'SEX_IM_Male', 'SPEEDREL_UPD_Racing',
       'SPEEDREL_UPD_Yes', 'TRAV_SP_55orLess', 'TRAV_SP_70orLess',
       'TRAV_SP_90Plus', 'TRAV_SP_90orLess', 'TRAV_SP_StoppedVehicle',
       'VE_TOTAL_One', 'VE_TOTAL_Three', 'VE_TOTAL_Two', 'VSPD_LIM_26-55',
       'VSPD_LIM_56-70', 'VSPD_LIM_71-90', 'VSPD_LIM_NonTraffic',
       'VSURCOND_Ice', 'VSURCOND_MudDirt', 'VSURCOND_NonTrafficway',
       'VSURCOND_Oil', 'VSURCOND_Other', 'VSURCOND_Sand', 'VSURCOND_Slush',
       'VSURCOND_Snow', 'VSURCOND_Water', 'VSURCOND_Wet', 'WEATHER_IM_Clear',
       'WEATHER_IM_Cloudy', 'WEATHER_IM_Fog', 'WEATHER_IM_Hail',
       'WEATHER_IM_Other', 'WEATHER_IM_Rain', 'WEATHER_IM_SevereCrosswinds',
       'WEATHER_IM_Snow', 'WEEKDAY_UPD_Monday', 'WEEKDAY_UPD_Saturday',
       'WEEKDAY_UPD_Sunday', 'WEEKDAY_UPD_Thursday', 'WEEKDAY_UPD_Tuesday',
       'WEEKDAY_UPD_Wednesday']

backward = ['ALCHL_IM_Yes', 'DAYNIGHT_Nighttime', 'DRUGS_Yes',
       'IMPACT1_IM_LeftFront', 'IMPACT1_IM_NonCollision', 'IMPACT1_IM_Rear',
       'IMPACT1_IM_Top', 'LGTCON_IM_Daylight', 'REST_TYP_NotApplicable',
       'REST_TYP_ShoulderLap', 'ROLLOVER_Yes', 'SEAT_IM_FrontR',
       'SEAT_IM_SecondL', 'SEAT_IM_SecondR', 'SEAT_IM_SleeperSection',
       'SPEEDREL_UPD_Yes', 'TRAV_SP_55orLess', 'TRAV_SP_90Plus',
       'TRAV_SP_90orLess', 'VSPD_LIM_56-70', 'VSURCOND_NonTrafficway',
       'WEATHER_IM_Snow', 'WEEKDAY_UPD_Saturday', 'WEEKDAY_UPD_Sunday']

forward = ['ALCHL_IM_Yes', 'DRUGS_Yes', 'IMPACT1_IM_NonCollision',
       'IMPACT1_IM_PartsinMotion', 'REST_TYP_NoRest', 'REST_TYP_NotApplicable',
       'REST_TYP_OtherRest', 'REST_TYP_ShoulderLap', 'ROLLOVER_Yes',
       'SEAT_IM_SleeperSection', 'SEAT_IM_TrailerorExterior',
       'TRAV_SP_55orLess', 'TRAV_SP_90Plus', 'TRAV_SP_90orLess',
       'VSPD_LIM_26-55', 'VSPD_LIM_56-70', 'VSURCOND_NonTrafficway',
       'VSURCOND_Sand', 'VSURCOND_Slush', 'VSURCOND_Snow', 'VSURCOND_Wet',
       'WEATHER_IM_Other', 'WEATHER_IM_Snow', 'WEEKDAY_UPD_Tuesday',
       'WEEKDAY_UPD_Wednesday']


# create a list comprehension for each smaller list
appearances_1 = [name for name in predictors1 if name in backward]
appearances_2 = [name for name in predictors1 if name in forward]

# find the names that appear in all three lists
best_predictors_pl = list(set(appearances_1) & set(appearances_2))

# print the results
print(f"The names that appear in all three lists are: {best_predictors_pl}")

The names that appear in all three lists are: ['VSURCOND_NonTrafficway', 'VSPD_LIM_56-70', 'DRUGS_Yes', 'TRAV_SP_90Plus', 'TRAV_SP_90orLess', 'IMPACT1_IM_NonCollision', 'REST_TYP_ShoulderLap', 'ALCHL_IM_Yes', 'ROLLOVER_Yes', 'SEAT_IM_SleeperSection', 'REST_TYP_NotApplicable', 'TRAV_SP_55orLess', 'WEATHER_IM_Snow']


# Final Model - Post Lockdown Period

In [46]:
# creation of the train and test data for the selection process for the before lockdown period. 
X_pl = df_post_lockdown.drop(columns = (['INJ_SEV_CAT', 'MONTH', 'YEAR']))
y_pl = df_post_lockdown['INJ_SEV_CAT']

x_trainpl, x_testpl, y_trainpl, y_testpl = train_test_split(X_pl,y_pl,test_size=0.3,random_state=5000)

In [47]:
## running the model
x_trainpl = x_trainpl[best_predictors_pl]

model = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=10000)
# define the model evaluation procedure
model.fit(x_trainpl,y_trainpl)

LogisticRegression(max_iter=10000, multi_class='multinomial')

In [48]:
# getting coefficients and p-values for analysis
coef = model.coef_
pval = 2*(1 - stats.norm.cdf(abs(coef))) # assuming a two-tailed test

# print coefficients and p-values
for i in range(len(x_trainpl.columns)):
    print(x_trainpl.columns[i], "Coefficient:", coef[0][i], "P-value:", pval[0][i])

VSURCOND_NonTrafficway Coefficient: 0.329838509480287 P-value: 0.7415219878248247
VSPD_LIM_56-70 Coefficient: -0.050903559294804335 P-value: 0.9594023692817719
DRUGS_Yes Coefficient: -0.38044178170314075 P-value: 0.703617504701529
TRAV_SP_90Plus Coefficient: -0.601689059857097 P-value: 0.5473811336405818
TRAV_SP_90orLess Coefficient: -0.4096508113657153 P-value: 0.6820621178521871
IMPACT1_IM_NonCollision Coefficient: -0.26027087855045755 P-value: 0.7946548342527904
REST_TYP_ShoulderLap Coefficient: -0.3205344535739763 P-value: 0.7485632167772585
ALCHL_IM_Yes Coefficient: -0.44061358174412685 P-value: 0.6594927686266332
ROLLOVER_Yes Coefficient: -1.3927731921077011 P-value: 0.16368838745326864
SEAT_IM_SleeperSection Coefficient: 1.9797763252098812 P-value: 0.04772866775409046
REST_TYP_NotApplicable Coefficient: -1.9730564152798644 P-value: 0.04848913520516307
TRAV_SP_55orLess Coefficient: -0.17984983048326497 P-value: 0.8572704623173646
WEATHER_IM_Snow Coefficient: 0.4643571702076572 P-

In [49]:
# create a pandas DataFrame with variable names, coefficients, and p-values
df = pd.DataFrame({'Variable': x_trainpl.columns, 'Coefficient': coef[0], 'P-value': pval[0]})

# print the DataFrame
print(df)

                   Variable  Coefficient   P-value
0    VSURCOND_NonTrafficway     0.329839  0.741522
1            VSPD_LIM_56-70    -0.050904  0.959402
2                 DRUGS_Yes    -0.380442  0.703618
3            TRAV_SP_90Plus    -0.601689  0.547381
4          TRAV_SP_90orLess    -0.409651  0.682062
5   IMPACT1_IM_NonCollision    -0.260271  0.794655
6      REST_TYP_ShoulderLap    -0.320534  0.748563
7              ALCHL_IM_Yes    -0.440614  0.659493
8              ROLLOVER_Yes    -1.392773  0.163688
9    SEAT_IM_SleeperSection     1.979776  0.047729
10   REST_TYP_NotApplicable    -1.973056  0.048489
11         TRAV_SP_55orLess    -0.179850  0.857270
12          WEATHER_IM_Snow     0.464357  0.642392


In [50]:
## to test the model accuracy
x_testpl = x_testpl[best_predictors_pl]

y_predictpl = model.predict(x_testpl)
accuracy_score(y_testpl,y_predictpl)

0.7170130688080617

In [51]:
## to test the model accuracy with the training data to avoid overfitting
y_predict_trainpl = model.predict(x_trainpl)
accuracy_score(y_trainpl,y_predict_trainpl)

0.7167513032915492

In [52]:
classificationSummary(y_testpl,y_predictpl, class_names=labels) 

Confusion Matrix (Accuracy 0.7170)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                    16936                483                 73
No severe injuries               5124               1035                138
Severe injuries                   792                579                244


In [53]:
# define class labels
labels = ['No injuries      ', 'No severe injuries', 'Severe injuries   ']

# create confusion matrix
cm = confusion_matrix(y_testpl, y_predictpl)

# print confusion matrix with labels
print('Confusion matrix:')
print(cm)

# print confusion matrix with labels and formatting
print('\nConfusion matrix (with labels):')
print('{:<19s}{:>20s}{:>15s}{:>20s}'.format('', *labels))
for i, label in enumerate(labels):
    print('{:<15s}{:>18d}{:>19d}{:>20d}'.format(label, *cm[i]))

Confusion matrix:
[[16936   483    73]
 [ 5124  1035   138]
 [  792   579   244]]

Confusion matrix (with labels):
                      No injuries      No severe injuries  Severe injuries   
No injuries                   16936                483                  73
No severe injuries              5124               1035                 138
Severe injuries                  792                579                 244


## Naive Bayes Model

In [54]:
# run naive Bayes
injury_nb = MultinomialNB(alpha=0.01)
injury_nb.fit(x_trainpl,y_trainpl)

# predict probabilities
predProb_train = injury_nb.predict_proba(x_trainpl)
predProb_valid = injury_nb.predict_proba(x_testpl)

# predict class membership
y_valid_pred = injury_nb.predict(x_testpl)
y_train_pred = injury_nb.predict(x_trainpl)

In [55]:
classificationSummary(y_trainpl, y_train_pred, class_names=labels) 

print()

classificationSummary(y_testpl, y_valid_pred, class_names=labels)

Confusion Matrix (Accuracy 0.7138)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                    39207               1345                129
No severe injuries              11843               2534                376
Severe injuries                  1952               1320                567

Confusion Matrix (Accuracy 0.7150)

                   Prediction
            Actual  No injuries       No severe injuries Severe injuries   
 No injuries                    16887                535                 70
No severe injuries               5121               1032                144
Severe injuries                   790                579                246
